In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

2025-07-28 13:47:59,071 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Carregar o arquivo CSV
df_times = pd.read_excel("times.xlsx")

# Ver os dados carregados
display(df_times.head(20))

,Nome
0,A Lenda Super Vascão f.c
1,A Lenda Super Vasco F.c
2,Analove10 ITAQUI GRANDE!!
3,BordonFC
4,BORGES ITAQUI F.C.
5,CALOMBO ITAQUI RS
6,cartola scheuer
7,clarinvalau fc
8,CosmoCity ZO
9,Cril Futebol Club


### Buscar IDs dos times no Cartola

In [3]:
# nomes_times = ['FBC Colorado', 'Texas Club 2025', 'Real SCI', 'Gig@ntte', 'PraiaBravaFC', 'OlhaEleAiF.C!', 'Gremiomaniasm', 'Sport Clube PAIM', 'PUXE FC', 'RS Expressões da Arte',
#                'ZIVI FC', 'O clube do povo Itaqui/Rss', 'F.C. Rei Das Copas', 'Rolo Compressor ZN', 'Taura da Fronteira FCIII', 'Eleis-Itaqui', 'KING LEONN', 'Laranjja Mecannica', 
#                'Fedato Futebol Clube', 'Perronee F.C', 'Pity10', 'pra sempre imortal fc', 'RHANKA DENTY FC25', 'TEAM LOPES 99', 'pura bucha /botafogo', 'cartola scheuer', 'Remo Santo Ângelo', 
#                'Analove10 ITAQUI GRANDE!!', 'DM Studio', 'lsauer fc', 'VASCO MARTINS FC', 'KP JUV.', 'BORGES ITAQUI F.C.', 'Profit Soccer', 'Tabajara de Inhaua FC2', 'TIGRE LEON', 
#                'S.E.R. GRILLO', 'seralex', 'E.C. Bororé', 'MAFRA MARTINS FC', 'BordonFC', 'Tatols Beants F.C', 'FIGUEIRA DA ILHA', 'MauHumor F.C.', 'A Lenda Super Vascão f.c', 
#                'TATITTA FC', 'HS SPORTS F.C', 'Dom Camillo68', 'mercearia Estrela', 'CosmoCity ZO', 'clarinvalau fc', 'Grêmio imortal 37', 'SERGRILLO', 'Super Vasco f.c', 
#                'A Lenda Super Vasco F.c', 'Paulo Virgili FC', 'CALOMBO ITAQUI RS', 'Luis lemes inter', 'emer jr fc', 'Cril Futebol Club', 'ITAQUI F.C.', 'TORRESMO COM PINGA', 
#                'Lá do Itaqui', 'FC Los Castilho', 'KillerColorado', "Atlético Colorado 2021", "Time do S.A.P.O", "SISO FC25", "Grêmio imortal 37", "BORGES ITAQUI F.C."]

nomes_times = ["PraiaBravaFC", "PUXE FC", "Texas Club 2025", "TIGRE LEON", "lsauer fc", "Real SCI", "Grêmio imortal 37", "cartola scheuer", "BORGES CLIMA FUT F.C", "Fedato Futebol Clube",
               "pura bucha /botafogo", "A Lenda Super Vasco F.c", "Pity10", "seralex", "Tatols Beants F.C", "Sport Clube PAIM", "HS SPORTS F.C", "Analove10 ITAQUI GRANDE!!", "FBC Colorado",
               "Dom Camillo68",]

# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função robusta para buscar ID do time por nome
from difflib import get_close_matches

def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)

        # 🛡️ Proteção caso a resposta seja uma string (ex: erro HTML ou mensagem)
        if not isinstance(times, list):
            print(f"⚠️ Resposta inesperada para '{nome_time}':", times)
            return None

        nomes_api = [time.nome for time in times]

        # Comparação flexível (removendo acentos, pontos, etc)
        nome_base = nome_time.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip()
        nomes_api_base = [n.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip() for n in nomes_api]

        nome_proximo = get_close_matches(nome_base, nomes_api_base, n=1, cutoff=0.6)

        if nome_proximo:
            idx = nomes_api_base.index(nome_proximo[0])
            return times[idx].id

    except Exception as e:
        print(f"❌ Erro ao buscar ID para o time '{nome_time}': {e}")
    return None


# Buscar IDs automaticamente, sem sobrescrever os que já estiverem no dicionário
nao_encontrados = []

for nome in nomes_times:
    if nome not in ids_times:  # Protege os manuais
        time_id = buscar_id_time(nome)
        if time_id:
            ids_times[nome] = time_id
        else:
            nao_encontrados.append(nome)

# Mostrar resultado
print("IDs encontrados:", len(ids_times))
print("Times não encontrados automaticamente:", nao_encontrados)

IDs encontrados: 20
Times não encontrados automaticamente: []


### Gerar DataFrame com nome do time, ID e URL

In [4]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_serie_B.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls.head(30))

✅ Arquivo salvo com sucesso: links_times_cartola_liga_serie_B.xlsx


,Nome do Time,ID do Time,Link do Time
0,PraiaBravaFC,47546604,https://cartola.globo.com/#!/time/47546604
1,PUXE FC,3447341,https://cartola.globo.com/#!/time/3447341
2,Texas Club 2025,1273719,https://cartola.globo.com/#!/time/1273719
3,TIGRE LEON,3424598,https://cartola.globo.com/#!/time/3424598
4,lsauer fc,44810918,https://cartola.globo.com/#!/time/44810918
5,Real SCI,3246608,https://cartola.globo.com/#!/time/3246608
6,Grêmio imortal 37,24468241,https://cartola.globo.com/#!/time/24468241
7,cartola scheuer,3851966,https://cartola.globo.com/#!/time/3851966
8,BORGES CLIMA FUT F.C,49852616,https://cartola.globo.com/#!/time/49852616
9,Fedato Futebol Clube,18642587,https://cartola.globo.com/#!/time/18642587


### Gerar o dicionário ID -> Nome do Time

In [5]:
# Gerar o dicionário ID -> Nome do Time
nomes_por_id = dict(zip(df_urls["ID do Time"], df_urls["Nome do Time"]))

# Mostrar parte do dicionário
dict(list(nomes_por_id.items())[:5])

{47546604: 'PraiaBravaFC',
 3447341: 'PUXE FC',
 1273719: 'Texas Club 2025',
 3424598: 'TIGRE LEON',
 44810918: 'lsauer fc'}

In [6]:
def campeonato_comecou(api, ids_times):
    """Verifica se o campeonato já começou observando a pontuação na 1ª rodada."""
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=1).ultima_pontuacao
            if pontuacao is not None:
                return True
        except cartolafc.errors.CartolaFCError:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """Obtém a pontuação do time em cada rodada até a rodada atual."""
    pontuacoes = {}
    for rodada in range(1, rodada_atual):
        try:
            time_rodada = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_rodada.ultima_pontuacao
        except cartolafc.errors.CartolaFCError as e:
            print(f"Erro ao acessar pontuação da rodada {rodada} para o time {time_id}: {e}")
            pontuacoes[rodada] = None
    return pontuacoes

def gerar_df_pontuacoes(api, ids_times):
    rodada_atual = api.mercado().rodada_atual
    total_rodadas = 38

    if not campeonato_comecou(api, ids_times):
        print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
        df = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
        df[:] = 0
    else:
        df = pd.DataFrame()
        for nome, time_id in ids_times.items():
            pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
            df[nome] = pd.Series(pontuacoes)
        df = df.transpose()
        df.columns = [f'Rodada {i}' for i in range(1, rodada_atual)]
        df.loc['Lider_Rodada'] = df.idxmax()
    
    return df

In [7]:
ids_times = {v: k for k, v in nomes_por_id.items()}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

display(df_pontuacoes.head(20))

,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Rodada 13,Rodada 14,Rodada 15,Rodada 16,Rodada 17
PraiaBravaFC,96.39,96.19,87.05,81.06,94.26,80.06,75.89,66.00,24.63,54.12,60.96,66.66,96.71,87.86,91.20,65.14,89.14
PUXE FC,84.69,95.29,86.95,85.01,73.92,88.76,96.11,103.93,77.90,61.47,93.21,58.84,128.04,74.30,101.50,79.97,53.21
Texas Club 2025,67.69,85.99,76.92,79.86,71.32,119.85,98.71,109.15,83.90,73.67,110.51,69.26,111.91,74.20,101.80,74.01,47.51
TIGRE LEON,69.40,83.04,83.60,61.83,62.41,85.16,100.16,117.55,69.48,47.87,71.71,93.51,113.67,75.80,103.16,69.21,51.98
lsauer fc,72.38,79.29,82.17,104.93,66.22,108.56,87.71,118.45,69.15,57.17,118.31,65.81,117.11,73.10,98.50,80.35,44.16
Real SCI,104.09,75.69,87.02,99.66,82.42,103.26,94.34,108.45,60.57,50.37,104.47,89.61,103.81,75.10,98.90,50.60,62.24
Grêmio imortal 37,61.55,99.39,82.42,122.26,63.40,88.36,91.51,131.35,59.77,60.77,122.41,65.86,133.05,80.90,81.20,50.60,80.10
cartola scheuer,73.09,86.39,75.10,77.36,80.92,98.30,95.79,99.45,73.27,73.57,108.04,69.26,132.21,74.20,101.80,70.17,52.84
BORGES CLIMA FUT F.C,64.09,93.19,62.90,85.16,68.42,106.15,111.86,150.95,95.92,69.27,114.54,62.26,96.51,64.80,91.60,76.66,36.89
Fedato Futebol Clube,77.58,77.49,75.42,90.16,81.02,92.06,104.31,98.49,62.97,66.07,110.41,56.26,107.65,78.38,88.49,85.05,58.25


### Função para definir a classificação dos times

In [8]:
def classificacao_por_grupo(df_rodadas, df_pontuacoes):
    """
    Classificação dos grupos com base nos confrontos e nas pontuações do Cartola.

    Retorna:
    - df_resultado: classificação geral
    - df_resultado_por_grupo: dicionário com classificação separada por grupo
    """
    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
    estatisticas = {}

    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        grupo = confronto["Grupo"]
        coluna_rodada = f"Rodada {rodada}"

        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue
        if coluna_rodada not in df_pontuacoes_times.columns:
            continue

        pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
        pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

        if pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante):
            continue

        for time in [mandante, visitante]:
            if grupo not in estatisticas:
                estatisticas[grupo] = {}
            if time not in estatisticas[grupo]:
                estatisticas[grupo][time] = {
                    "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                    "Total_Cartola": 0, "Cartola_Sofrido": 0
                }

        # Atualizar estatísticas do jogo
        estatisticas[grupo][mandante]["Total_Cartola"] += pontos_mandante
        estatisticas[grupo][mandante]["Cartola_Sofrido"] += pontos_visitante

        estatisticas[grupo][visitante]["Total_Cartola"] += pontos_visitante
        estatisticas[grupo][visitante]["Cartola_Sofrido"] += pontos_mandante

        if pontos_mandante > pontos_visitante:
            estatisticas[grupo][mandante]["Pontos"] += 3
            estatisticas[grupo][mandante]["Vitórias"] += 1
            estatisticas[grupo][visitante]["Derrotas"] += 1
        elif pontos_mandante < pontos_visitante:
            estatisticas[grupo][visitante]["Pontos"] += 3
            estatisticas[grupo][visitante]["Vitórias"] += 1
            estatisticas[grupo][mandante]["Derrotas"] += 1
        else:
            estatisticas[grupo][mandante]["Pontos"] += 1
            estatisticas[grupo][visitante]["Pontos"] += 1
            estatisticas[grupo][mandante]["Empates"] += 1
            estatisticas[grupo][visitante]["Empates"] += 1

    # Montar DataFrame final
    df_resultado = pd.concat([
        pd.DataFrame({
            "Grupo": grupo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
            "Saldo Cartola": [stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
            ]
        })
        for grupo, times in estatisticas.items()
    ], ignore_index=True)


    df_resultado = df_resultado.sort_values(
        by=["Grupo", "Pontos", "Total Cartola"],
        ascending=[True, False, False]
    )

    df_resultado["Posição"] = df_resultado.groupby("Grupo")\
    .cumcount() + 1

    df_resultado_por_grupo = {
        grupo: df_resultado[df_resultado["Grupo"] == grupo] for grupo in df_resultado["Grupo"].unique()
    }

    return df_resultado, df_resultado_por_grupo

In [9]:
# 1. Carregar confrontos
df_confrontos = pd.read_csv("confrontos_serie_B.csv")

# 2. Renomear colunas se necessário
df_confrontos.rename(columns={
    "Time A": "Mandante_Nome",
    "Time B": "Visitante_Nome"
}, inplace=True)

# 3. Adicionar coluna de grupo fixo
df_confrontos["Grupo"] = "Série B"

display(df_confrontos.head())

# 4. Calcular classificação
df_classificacao, _ = classificacao_por_grupo(df_confrontos, df_pontuacoes)

# 5. Ver resultado
display(df_classificacao.head(20))

,Rodada,Confronto,Mandante_Nome,Visitante_Nome,ID A,ID B,Grupo
0,1,1,lsauer fc,FBC Colorado,18,5,Série B
1,1,2,Real SCI,A Lenda Super Vasco F.c,12,1,Série B
2,1,3,PUXE FC,Analove10 ITAQUI GRANDE!!,9,2,Série B
3,1,4,Fedato Futebol Clube,cartola scheuer,6,17,Série B
4,1,5,PraiaBravaFC,Sport Clube PAIM,11,13,Série B


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
10,Série B,Texas Club 2025,37,12,1,4,1456.26,1277.92,178.34,1
4,Série B,PUXE FC,36,12,0,5,1443.10,1353.53,89.57,2
11,Série B,Grêmio imortal 37,33,11,0,6,1474.90,1468.03,6.87,3
5,Série B,Analove10 ITAQUI GRANDE!!,30,10,0,7,1529.54,1428.28,101.26,4
0,Série B,lsauer fc,30,10,0,7,1443.37,1401.09,42.28,5
18,Série B,Pity10,30,10,0,7,1335.78,1434.10,-98.32,6
1,Série B,FBC Colorado,27,9,0,8,1448.58,1408.31,40.27,7
6,Série B,Fedato Futebol Clube,27,9,0,8,1410.06,1357.58,52.48,8
17,Série B,Tatols Beants F.C,27,9,0,8,1381.59,1419.42,-37.83,9
3,Série B,A Lenda Super Vasco F.c,25,8,1,8,1447.59,1464.47,-16.88,10


In [10]:
# 1. Times únicos nos confrontos
times_confrontos = pd.unique(df_confrontos[["Mandante_Nome", "Visitante_Nome"]].values.ravel())

# 2. Times únicos na classificação final
times_classificados = df_classificacao["Nome do Time"].unique()

# 3. Ver quem está nos confrontos mas não foi classificado
faltando = set(times_confrontos) - set(times_classificados)

print("❌ Times que estão nos confrontos, mas faltam na classificação:")
print(faltando)

# df_pontuacoes.loc[["Time do S.A.P.O", "BORGES ITAQUI F.C."]]


❌ Times que estão nos confrontos, mas faltam na classificação:
set()


In [11]:
# Exportar resultado em CSV
df_classificacao.to_csv("classificacao_serie_B.csv", index=False)

# Carregar o arquivo CSV enviado
df_classificacao = pd.read_csv("classificacao_serie_B.csv")

# Renomear colunas para os nomes usados no JavaScript
df_classificacao.rename(columns={
    "Grupo": "grupo",
    "Nome do Time": "nome",
    "Pontos": "pontos",
    "Vitórias": "vitorias",
    "Empates": "empates",
    "Derrotas": "derrotas",
    "Total Cartola": "totalCartola",
    "Cartola Sofrido": "cartolaSofrido",
    "Saldo Cartola": "saldoCartola",
    "Posição": "posicao"
}, inplace=True)

# Agrupar por grupo (apenas Série A nesse caso)
classificacao_por_grupo = {}
for grupo, dados in df_classificacao.groupby("grupo"):
    classificacao_por_grupo[grupo] = dados.drop(columns="grupo").to_dict(orient="records")

# Salvar como arquivo JavaScript
caminho_saida = "classificacao_serie_B.js"
with open(caminho_saida, "w", encoding="utf-8") as f:
    f.write(f"const classificacaoSerieA = {json.dumps(classificacao_por_grupo, indent=2, ensure_ascii=False)};")

caminho_saida

'classificacao_serie_B.js'

In [12]:
import pandas as pd
import json

# Carregar CSV
df_confrontos = pd.read_csv("confrontos_serie_B.csv")

# Limpar espaços nas colunas
df_confrontos.columns = df_confrontos.columns.str.strip()

# Renomear colunas
df_confrontos.rename(columns={
    "Rodada": "rodada",
    "Confronto": "confronto",
    "Time A": "mandante_nome",
    "Time B": "visitante_nome",
    "ID A": "mandante_id",
    "ID B": "visitante_id"
}, inplace=True)

# Adicionar grupo fixo
df_confrontos["grupo"] = "Série B"

# Gerar estrutura para JS
confrontos_formatado = []
for _, row in df_confrontos.iterrows():
    confronto = {
        "rodada": int(row["rodada"]),
        "confronto": int(row["confronto"]),
        "grupo": row["grupo"],
        "mandante": {"id": row["mandante_id"], "nome": row["mandante_nome"]},
        "visitante": {"id": row["visitante_id"], "nome": row["visitante_nome"]}
    }
    confrontos_formatado.append(confronto)

# Salvar como arquivo JS
js_output = f"const confrontosFase1 = {json.dumps(confrontos_formatado, indent=2, ensure_ascii=False)};"
with open("confrontos_serie_B.js", "w", encoding="utf-8") as f:
    f.write(js_output)

print("✅ Arquivo confrontos_serie_B.js gerado com sucesso!")


✅ Arquivo confrontos_serie_B.js gerado com sucesso!


In [13]:
# Carregar o arquivo CSV enviado
df_confrontos = pd.read_csv("confrontos_serie_B.csv")

display(df_confrontos.head())

,Rodada,Confronto,Time A,Time B,ID A,ID B
0,1,1,lsauer fc,FBC Colorado,18,5
1,1,2,Real SCI,A Lenda Super Vasco F.c,12,1
2,1,3,PUXE FC,Analove10 ITAQUI GRANDE!!,9,2
3,1,4,Fedato Futebol Clube,cartola scheuer,6,17
4,1,5,PraiaBravaFC,Sport Clube PAIM,11,13


In [14]:
def gerar_resultados_serie_a(df_confrontos, df_pontuacoes, grupo="Série B"):
    """
    Gera um DataFrame com os resultados dos confrontos da Série A por rodada.
    """
    resultados = []

    for _, row in df_confrontos.iterrows():
        rodada = row["Rodada"]
        mandante = row["Time A"]
        visitante = row["Time B"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "grupo": grupo,
            "rodada": rodada,
            "mandante": {
                "nome": mandante,
                "pontos": pontos_mandante
            },
            "visitante": {
                "nome": visitante,
                "pontos": pontos_visitante
            }
        })

    return pd.DataFrame(resultados)


In [15]:
df_resultados = gerar_resultados_serie_a(df_confrontos, df_pontuacoes)

# Exportar para .js
import json

with open("resultados_serie_B.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase1 = ")
    f.write(json.dumps(df_resultados.to_dict(orient="records"), indent=2, ensure_ascii=False))
    f.write(";")

print("✅ Arquivo resultados_serie_B.js gerado com sucesso.")


✅ Arquivo resultados_serie_B.js gerado com sucesso.
